# 📖 Notebook 1 - Ingestion et Nettoyage des Textes Harry Potter

Ce notebook extrait et nettoie les textes des livres Harry Potter depuis les PDFs.

## Objectifs
1. Extraire le texte brut des 7 livres PDF
2. Nettoyer et normaliser le texte
3. Segmenter par phrases et dialogues
4. Exporter en format parquet pour analyse ultérieure

## Sorties
- `data/sentences.parquet` : corpus segmenté par phrases
- `data/book_metadata.json` : métadonnées des livres

In [ ]:
# Imports
import os
import re
import json
from pathlib import Path
from typing import Dict, List, Tuple
import PyPDF2
import pandas as pd
import numpy as np
from tqdm import tqdm

# Configuration des chemins
NOTEBOOK_DIR = Path().absolute()
PROJECT_ROOT = NOTEBOOK_DIR.parent
BOOKS_DIR = PROJECT_ROOT / "../../context/books"
DATA_DIR = NOTEBOOK_DIR.parent / "data"

# Créer le dossier data si nécessaire
DATA_DIR.mkdir(exist_ok=True)

print(f"📁 Project root: {PROJECT_ROOT}")
print(f"📚 Books directory: {BOOKS_DIR}")
print(f"💾 Data output: {DATA_DIR}")

In [ ]:
# Métadonnées des livres
BOOK_INFO = {
    "harry-potter-1-lecole-des-sorciers.pdf": {
        "title": "L'École des Sorciers",
        "book_number": 1,
        "pages": 320,
        "year": 1997
    },
    "harry-potter-2-la-chambre-des-secrets.pdf": {
        "title": "La Chambre des Secrets",
        "book_number": 2,
        "pages": 360,
        "year": 1998
    },
    "harry-potter-3-le-prisonnier-dazkaban.pdf": {
        "title": "Le Prisonnier d'Azkaban",
        "book_number": 3,
        "pages": 420,
        "year": 1999
    },
    "harry-potter-4-la-coupe-de-feu.pdf": {
        "title": "La Coupe de Feu",
        "book_number": 4,
        "pages": 656,
        "year": 2000
    },
    "harry-potter-5-lordre-du-phoenix.pdf": {
        "title": "L'Ordre du Phénix",
        "book_number": 5,
        "pages": 980,
        "year": 2003
    },
    "harry-potter-6-le-prince-de-sang-mecc82lecc81.pdf": {
        "title": "Le Prince de Sang-Mêlé",
        "book_number": 6,
        "pages": 640,
        "year": 2005
    },
    "harry-potter-7-les-reliques-de-la-mort.pdf": {
        "title": "Les Reliques de la Mort",
        "book_number": 7,
        "pages": 800,
        "year": 2007
    }
}

print(f"📚 Nombre de livres configurés: {len(BOOK_INFO)}")

## 1. Extraction du texte depuis les PDFs

In [ ]:
def extract_text_from_pdf(pdf_path: Path) -> str:
    """Extrait le texte d'un fichier PDF."""
    text = ""
    
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            print(f"  📄 Pages: {len(pdf_reader.pages)}")
            
            for page_num, page in enumerate(tqdm(pdf_reader.pages, desc="Extraction")):
                try:
                    text += page.extract_text() + "\n"
                except Exception as e:
                    print(f"  ⚠️  Erreur page {page_num}: {e}")
                    continue
    except Exception as e:
        print(f"  ❌ Erreur lecture PDF: {e}")
        return ""
    
    return text


def clean_text(text: str) -> str:
    """Nettoie le texte extrait."""
    # Supprimer les sauts de ligne excessifs
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    # Supprimer les espaces multiples
    text = re.sub(r' +', ' ', text)
    
    # Normaliser les guillemets
    text = text.replace('\u2018', "'").replace('\u2019', "'")
    text = text.replace('\u201c', '«').replace('\u201d', '»')
    
    return text.strip()

In [ ]:
# Extraire le texte de tous les livres
books_data = []

for filename, info in BOOK_INFO.items():
    pdf_path = BOOKS_DIR / filename
    
    if not pdf_path.exists():
        print(f"⚠️  Livre non trouvé: {filename}")
        continue
    
    print(f"\n📖 Extraction: {info['title']}")
    
    # Extraire et nettoyer
    raw_text = extract_text_from_pdf(pdf_path)
    clean = clean_text(raw_text)
    
    # Statistiques de base
    word_count = len(clean.split())
    char_count = len(clean)
    
    print(f"  ✅ Mots: {word_count:,}")
    print(f"  ✅ Caractères: {char_count:,}")
    
    books_data.append({
        'filename': filename,
        'book_number': info['book_number'],
        'title': info['title'],
        'pages': info['pages'],
        'year': info['year'],
        'text': clean,
        'word_count': word_count,
        'char_count': char_count
    })

print(f"\n✅ Total livres extraits: {len(books_data)}")

## 2. Segmentation en phrases

In [ ]:
def segment_into_sentences(text: str) -> List[str]:
    """Segmente le texte en phrases.
    
    Note: Pour une segmentation plus précise, on utilisera spaCy dans le notebook 02.
    Ici, on fait une segmentation simple basée sur les points.
    """
    # Regex simple pour diviser sur . ! ? suivis d'espace et majuscule
    sentences = re.split(r'(?<=[.!?])\s+(?=[A-ZÀ-Ü])', text)
    
    # Filtrer les phrases trop courtes
    sentences = [s.strip() for s in sentences if len(s.strip()) > 10]
    
    return sentences

In [ ]:
# Créer le corpus de phrases
all_sentences = []

for book in books_data:
    print(f"\n📝 Segmentation: {book['title']}")
    
    sentences = segment_into_sentences(book['text'])
    
    for idx, sentence in enumerate(sentences):
        all_sentences.append({
            'book_number': book['book_number'],
            'book_title': book['title'],
            'sentence_id': idx,
            'text': sentence,
            'length': len(sentence)
        })
    
    print(f"  ✅ {len(sentences):,} phrases extraites")

print(f"\n✅ Total phrases: {len(all_sentences):,}")

## 3. Export des données

In [ ]:
# Créer DataFrame et exporter en parquet
df_sentences = pd.DataFrame(all_sentences)

print("\n📊 Aperçu du DataFrame:")
print(df_sentences.head())
print(f"\n📏 Shape: {df_sentences.shape}")
print(f"\n📈 Statistiques:")
print(df_sentences.groupby('book_title')['sentence_id'].count())

In [ ]:
# Exporter en parquet
output_path = DATA_DIR / 'sentences.parquet'
df_sentences.to_parquet(output_path, index=False)
print(f"✅ Données exportées: {output_path}")
print(f"📦 Taille fichier: {output_path.stat().st_size / 1024 / 1024:.2f} MB")

In [ ]:
# Exporter les métadonnées
metadata = {
    'books': [
        {
            'book_number': book['book_number'],
            'title': book['title'],
            'pages': book['pages'],
            'year': book['year'],
            'word_count': book['word_count'],
            'char_count': book['char_count']
        }
        for book in books_data
    ],
    'total_sentences': len(all_sentences),
    'extraction_date': pd.Timestamp.now().isoformat()
}

metadata_path = DATA_DIR / 'book_metadata.json'
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"✅ Métadonnées exportées: {metadata_path}")

## 4. Validation des données

In [ ]:
# Vérifier que les données peuvent être relues
df_test = pd.read_parquet(output_path)
print(f"✅ Vérification lecture parquet: {df_test.shape}")

# Afficher quelques exemples
print("\n📖 Exemples de phrases:")
for i in range(min(5, len(df_test))):
    row = df_test.iloc[i]
    print(f"\nLivre {row['book_number']} - {row['book_title']}")
    print(f"  {row['text'][:200]}...")

## ✅ Résumé

Ce notebook a:
1. ✅ Extrait le texte des 7 livres Harry Potter
2. ✅ Nettoyé et normalisé le texte
3. ✅ Segmenté en phrases
4. ✅ Exporté en format parquet

**Prochaine étape**: Notebook 02 - Pipeline NLP pour NER, coréférence et attribution de locuteur